KNN MODEL FOR CLASSIFICATION

Install packages

In [ ]:
# print(0)

In [ ]:
# ! pip install tensorflow
# ! pip install pandas

Data Pre-processing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
from scipy.stats import pearsonr

In [ ]:
# file_path = "TrainDataset2023.xls"
# # file_path = "TrainDataset2023.xls"

# df = pd.read_excel(file_path)

In [ ]:
# Loaded dataset and pickle
file_path = "TrainDataset2023.csv"
df=pd.read_csv(file_path)

In [ ]:
# Loaded dataset and pickle
# file_path = "TrainDataset2023.csv"
# df=pd.read_csv(file_path)
df.head()
target_lable="pCR (outcome)"
# model_path="model/cls_ann"
# pickle.load()

In [ ]:
df

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.impute import KNNImputer

# # # 创建一个包含'ID'列的示例DataFrame，以及其他可能需要填充的列
# # data = {'ID': [1, 2, 3, 4],
# #         'A': [1, 2, np.nan, 4],
# #         'B': [4, 5, 6, 7],
# #         'C': [7, np.nan, 9, 10]}

# # df = pd.DataFrame(data)

# # 将'ID'列保存到一个单独的Series中
# id_column = df['ID']

# # 从DataFrame中移除'ID'列，以便进行填充
# df.drop(columns=['ID'], inplace=True)

# # 初始化KNNImputer，指定n_neighbors
# knn_imputer = KNNImputer(missing_values='NaN',n_neighbors=1)

# # 执行KNN填充
# imputed_data = knn_imputer.fit_transform(df)

In [ ]:
import pandas as pd
import numpy as np


# 使用isna()方法检查每一列是否包含NaN值
nan_columns = df.columns[df.isna().any()].tolist()

# 输出包含NaN值的列
print("包含NaN值的列:", nan_columns)

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer

# # 创建一个示例数据集，包含NaN值
# data = np.array([[1, 2, np.nan],
#                  [4, np.nan, 6],
#                  [7, 8, 9],
#                  [np.nan, 11, 12]])

# 初始化KNNImputer，指定n_neighbors
knn_imputer = KNNImputer(n_neighbors=2)

# 使用KNN imputer填充NaN值
imputed_data = knn_imputer.fit_transform(df)

# 输出填充后的数据
print("Imputed Data:")
print(imputed_data)


In [ ]:
import pandas as pd
import numpy as np


# 使用isna()方法检查每一列是否包含NaN值
nan_columns = df.columns[df.isna().any()].tolist()

# 输出包含NaN值的列
print("包含NaN值的列:", nan_columns)


In [ ]:
import numpy as np
import pandas as pd

# 创建一个包含字符串'nan'的示例DataFrame
data = {'A': [1, 'nan', 3, 4],
        'B': ['nan', 6, 'nan', 8]}

df = pd.DataFrame(data)

# 将字符串'nan'转换为np.nan
df = df.apply(lambda x: np.where(x == 'nan', np.nan, x))

# 输出转换后的DataFrame
print(df)


In [ ]:
# # 将字符串'nan'转换为np.nan
# df = df.apply(lambda x: np.where(x == '999', np.nan, x))

# # 输出转换后的DataFrame
# print(df)

In [ ]:
# 初始化KNNImputer，指定n_neighbors
knn_imputer = KNNImputer(missing_values='NaN',n_neighbors=1)

# 执行KNN填充
imputed_data = knn_imputer.fit_transform(df)

In [ ]:
# 使用isna()方法检查每一列是否包含NaN值
nan_columns = df.columns[df.isna().any()].tolist()

# 输出包含NaN值的列
print("包含NaN值的列:", nan_columns)

In [ ]:
# 将填充后的数据与'ID'列重新组合
imputed_df = pd.DataFrame(imputed_data, columns=df.columns)
imputed_df['ID'] = id_column

# 输出填充后的DataFrame
print("Imputed DataFrame:")
print(imputed_df)


In [ ]:
# import pandas as pd
# import numpy as np


# # 使用isna()方法检查每一列是否包含NaN值
# nan_columns = df.columns[df.isna().any()].tolist()

# # 输出包含NaN值的列
# print("包含NaN值的列:", nan_columns)

In [ ]:
# import numpy as np
# from sklearn.impute import KNNImputer

# # # Create a sample dataset with missing values
# # data = np.array([[1, 2, np.nan],
# #                  [4, 5, 6],
# #                  [7, np.nan, 9],
# #                  [10, 11, 12]])

# # Initialize the KNNImputer with the desired number of neighbors (k)
# knn_imputer = KNNImputer(n_neighbors=2)

# # Perform KNN imputation
# imputed_data = knn_imputer.fit_transform(df)

# # The missing values have been filled in using KNN imputation
# print("Imputed Data:")
# print(imputed_data)


In [ ]:
# import pandas as pd
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(missing_values='NaN',strategy='mean')
# df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# df_imputed

In [ ]:
# df_imputed

In [ ]:
# X = df.drop(columns=[target_lable, 'RelapseFreeSurvival (outcome)'])
# # X = df.drop(columns=[target_lable, 'RelapseFreeSurvival (outcome)',"ID"])
# # X

# Assign features to X
X = df.drop(columns=['ID',target_lable, 'RelapseFreeSurvival (outcome)'], axis=1)
# X = df.drop(columns=[target_lable, 'RelapseFreeSurvival (outcome)'], axis=1,inplace=True)
# X=None
X

In [ ]:
# Data imputation 
# Use regression model to do data imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
impute_it = IterativeImputer()
impute_it.fit_transform(X)

In [ ]:
# Data imputation
# Use KNN model to do data imputation
from sklearn.impute import KNNImputer
impute_knn = KNNImputer(n_neighbors=2)
impute_knn.fit_transform(X)

In [ ]:
X

In [ ]:
# Identify the outcome
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[target_lable] = le.fit_transform(df[target_lable])
df.head()

# assign numerical label to y
y = df[target_lable]
y

Feature Selection

In [ ]:
# Normalise the features using zero mean normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
Xs_df = pd.DataFrame(Xs, columns=X.columns)
Xs_df['ID'] = df["ID"]

In [ ]:
# import pandas as pd
# from sklearn.impute import SimpleImputer

# # Data Preprocessing
# def preprocess_data(df):
#     # Imputing missing values
#     imputer = SimpleImputer(missing_values='NaN',strategy='mean')
   
#     df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

#     # Separating features and target label
#     X = df_imputed.drop('pCR (outcome)', axis=1)
#     y = df_imputed['pCR (outcome)']

#     # Normalizing the data
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     return train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
import pandas as pd
import numpy as np


# 使用isna()方法检查每一列是否包含NaN值
nan_columns = df.columns[df.isna().any()].tolist()

# 输出包含NaN值的列
print("包含NaN值的列:", nan_columns)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs_df, y, test_size=0.3, random_state=42)

In [ ]:
from tensorflow import keras
# from tensorflow.keras import layers
# # set ANN model
# model = keras.Sequential([
#     layers.Dense(64, activation='relu', input_shape=(X_train.shape[1] -1,)),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(1)  # Output layer for regression, no activation function
# ])
# then i compiled the model
# model.compile(optimizer='adam', loss='mean_absolute_error')
keras.fit(X_train.loc[:, X_train.columns != 'ID'], y_train, epochs=50, batch_size=32, validation_split=0.3)



In [ ]:
# evaluated the model on the test set
mae = model.evaluate(X_test.loc[:, X_test.columns != 'ID']
, y_test)
print(f'Mean Absolute Error (MAE): {mae}')

# predictions
y_pred = model.predict(X_test.loc[:, X_test.columns != 'ID'])

In [ ]:
y_pred.shape


In [ ]:
data = { 'ID': X_test["ID"], 'Prediction': y_pred.reshape((y_pred.shape[0],))}
final_df = pd.DataFrame(data)

In [ ]:
final_df